In [1]:
import setup

setup.init_django()

In [2]:
from rag import (
    db as rag_db,
    engines as rag_engines,
    settings as rag_settings,
    updaters as rag_updaters,
    sync as rag_sync, 
    patches as rag_patches,
)


In [3]:
from typing import Optional, Union
from sqlalchemy import create_engine, text

In [4]:
rag_settings.init()
rag_db.init_vector_db()
rag_sync.full_sync()

Starting sync for 12 documents across all models
Full synchronization completed


In [5]:
employee_index = rag_engines.get_model_index("Employee")
product_index = rag_engines.get_model_index("Product")
semantic_query_engine = rag_engines.get_semantic_query_engine("Product")  
sql_query_engine = rag_engines.get_sql_query_engine()

In [6]:
print(rag_settings.VECTOR_DB_NAME, rag_settings.VECTOR_DB_TABLE_NAMES)

vector_db ['employeeroles', 'employees', 'producttypes', 'products', 'inventoryitems', 'productinventoryrequirements']


In [7]:
from llama_index.core.tools import QueryEngineTool

vector_tool = QueryEngineTool.from_defaults(
    query_engine=semantic_query_engine,
    description=(
        "Useful for answering semantic questions about coffee shop operations, "
        "including products, employees, inventory, and recipes"
    ),
)


In [8]:
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over "
        "tables containing:EmployeeRole, Employee, ProductType, Product, InventoryItem, ProductInventoryRequirement."
    ),
)

In [9]:
query_engine = rag_patches.MySQLAutoVectorQueryEngine(
    sql_tool,
    vector_tool,
)

In [10]:
response = query_engine.query(
    "how mush is an late?"
)

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.sql_join_query_engine:> Querying other query engine: The question 'how mush is an americano?' pertains to coffee shop operations, specifically about products, which is covered by choice (2).


Querying other query engine: The question 'how mush is an americano?' pertains to coffee shop operations, specifically about products, which is covered by choice (2).


INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


In [11]:
from IPython.display import Markdown, display

display(Markdown(response.response))

'An Americano costs $2.50.'

In [12]:
response = query_engine.query(
    "give me the list of all employee"
)

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.sql_join_query_engine:> Querying SQL database: The question asks for a list of employees, and choice (1) mentions 'Employee' as one of the tables it can query.


Querying SQL database: The question asks for a list of employees, and choice (1) mentions 'Employee' as one of the tables it can query.


INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'coffeshop_employeerole' has columns: id (BIGINT), name (VARCHAR(100)), description (TEXT), embedding (VECTOR(1024)), .

Table 'coffeshop_employee' has columns: id (BIGINT), name (VARCHAR(200)), hire_date (DATE), hourly_rate (NUMERIC(6, 2)), is_active (BOOLEAN), embedding (VECTOR(1024)), role_id (BIGINT),  and foreign keys: ['role_id'] -> coffeshop_employeerole.['id'].

Table 'coffeshop_producttype' has columns: id (BIGINT), name (VARCHAR(100)), description (TEXT), embedding (VECTOR(1024)), .

Table 'coffeshop_product' has columns: id (BIGINT), name (VARCHAR(200)), price (NUMERIC(6, 2)), description (TEXT), is_available (BOOLEAN), embedding (VECTOR(1024)), product_type_id (BIGINT), last_indexed (TIMESTAMP),  and foreign keys: ['product_type_id'] -> coffeshop_producttype.['id'].

Table 'coffeshop_inventoryitem' has columns: id (BIGINT), name (VARCHAR(200)), unit (VARCHAR(50)), quantity (NUMERIC(10, 2)), reo

SQL query: SELECT coffeshop_employee.name, coffeshop_employee.hire_date, coffeshop_employee.hourly_rate, coffeshop_employee.is_active, coffeshop_employeerole.name AS role_name
FROM coffeshop_employee
JOIN coffeshop_employeerole ON coffeshop_employee.role_id = coffeshop_employeerole.id
ORDER BY coffeshop_employee.hire_date DESC;
SQL response: Here is the list of all employees, ordered by their hire date from newest to oldest:

1. **Harry West**
   - Hire Date: March 26, 2025
   - Hourly Rate: $35.00
   - Role: Barista
   - Status: Active

2. **John Doe**
   - Hire Date: March 23, 2025
   - Hourly Rate: $40.00
   - Role: Barista
   - Status: Active

3. **Mayra Anderson**
   - Hire Date: February 25, 2025
   - Hourly Rate: $40.00
   - Role: Cashier
   - Status: Active


INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.sql_join_query_engine:> Transformed query given SQL response: None


Transformed query given SQL response: None


In [13]:
from IPython.display import Markdown, display

display(Markdown(response.response))

Here is the list of all employees, ordered by their hire date from newest to oldest:

1. **Harry West**
   - Hire Date: March 26, 2025
   - Hourly Rate: $35.00
   - Role: Barista
   - Status: Active

2. **John Doe**
   - Hire Date: March 23, 2025
   - Hourly Rate: $40.00
   - Role: Barista
   - Status: Active

3. **Mayra Anderson**
   - Hire Date: February 25, 2025
   - Hourly Rate: $40.00
   - Role: Cashier
   - Status: Active

In [17]:
response = query_engine.query(
    "لیست تمام کارکنان"
)

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.sql_join_query_engine:> Querying other query engine: The question involves adding a new product to the database, which is related to inventory and products, making choice (2) more relevant.


Querying other query engine: The question involves adding a new product to the database, which is related to inventory and products, making choice (2) more relevant.


INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


In [18]:
from IPython.display import Markdown, display

display(Markdown(response.response))

To add a new product to the database, use the following details:

- **model_type:** Product
- **name:** icecream
- **price:** Not specified
- **description:** Everything that serves with only ice cream

In [21]:
response = query_engine.query(
    "averge price of all products"
)

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.sql_join_query_engine:> Querying other query engine: The question involves updating the price of a product, which is related to inventory and products in a coffee shop operation.


Querying other query engine: The question involves updating the price of a product, which is related to inventory and products in a coffee shop operation.


INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


In [22]:
from IPython.display import Markdown, display

display(Markdown(response.response))

I'm unable to update the price of the "icecream" product as there is no such product listed. The products available are "ice americano" and "latte".